In [17]:
import os
import time
import torch
import argparse
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from collections import defaultdict
import torch.multiprocessing as mp
import torch.utils.data
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
from sklearn import preprocessing


sides_3 = np.load('../data/force_torque_sensor/Dataset/3_sides/Data/data.npy')

dataset = sides_3[:,0:10]

device = torch.device('cuda:0')
print(type(dataset))

<class 'numpy.ndarray'>


In [18]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std

def un_normalize(normalized_data, input_data):
    mu = np.mean(input_data,axis=0)
    std = np.std(input_data,axis=0)
    return normalized_data*std+mu

In [19]:
def extract_trials(x, n):
    x = feature_normalize(x)
#     x = preprocessing.minmax_scale(x, feature_range=(0,1))
    trials=[]
    trial=[]
    compare=0
    for sig in x:
        trial.append(sig[0:n])
        if sig[11] <= compare:
            d = 1002 - len(trial)
            if d >0:
                for i in range(d):
                    trial.append(np.zeros(len(sig[0:n])))
            trials.append(trial)
            trial=[]
        compare=sig[11]
    trials = np.array(trials)
    return trials       


In [20]:
num=10

sides_3_trials = extract_trials(sides_3, num)

In [5]:
sides_3_trials = sides_3_trials[0:1530]
dataset = sides_3_trials

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 10

test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)


In [6]:
def loss_fn(recon_x, x, mean, log_var):
    BCE = torch.nn.functional.binary_cross_entropy(recon_x.view(-1, 1002*10), x.view(-1, 1002*10), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    # elbo = recon_loss + self.beta * D_kl
    elbo = (BCE + KLD)/ x.size(0)
    return elbo

In [22]:
vae = torch.load('vae_all_signals_1002.pkl').to(device)

import torch.utils.data


recon_signals=[]
latent_space=[]

for iteration, test in enumerate(test_loader):
    test = test.view(test.size(0), -1)
    test = test.to(device).float()
    test_recon_x, test_mean, test_log_var, test_z = vae(test)
    test_recon_x = test_recon_x.view(batch_size, 10, 1002)
    test_recon_x=test_recon_x.cpu().detach().numpy()
    latent_space=test_z.cpu().detach().numpy()

    recon_signals.extend(test_recon_x)
#     latent_space.extend(latent_space)
    
    if iteration ==10:
        break
    
recon_signals = np.array(recon_signals)
latent_space = np.array(latent_space)

In [23]:
print(type(recon_signals[0]))
print(recon_signals[0])

print(len(recon_signals))
print(len(recon_signals[0]))
# print(recon_signals)

<class 'numpy.ndarray'>
[[4.74400842e-27 9.93749857e-01 5.45790086e-21 ... 5.01516407e-20
  8.63500474e-27 9.92110670e-01]
 [3.99142514e-21 9.99999404e-01 9.93612111e-01 ... 9.94656682e-01
  1.69427168e-21 9.99999046e-01]
 [9.23313439e-01 5.44160068e-01 1.00000000e+00 ... 9.99999404e-01
  6.15298390e-01 5.59923540e-14]
 ...
 [1.00000000e+00 2.45404051e-22 2.85306710e-20 ... 9.99999166e-01
  1.00000000e+00 2.71395162e-22]
 [2.19585773e-20 3.14196885e-01 9.80087277e-03 ... 4.56931662e-22
  2.51889857e-20 2.82902002e-01]
 [2.78451771e-01 4.63286400e-01 5.10936195e-27 ... 1.10090755e-01
  1.87678216e-03 2.80768784e-20]]
110
10


In [24]:
for i in range(len(recon_signals[0])):
    recon_signals[:,i]=preprocessing.minmax_scale(
        recon_signals[:,i], feature_range=(np.min(sides_3[:,i]),np.max(sides_3[:,i])))


# recon_signals = un_normalize(recon_signals, sides_3_trials)
generate_signals = recon_signals


In [25]:
print(generate_signals)
    

[[[-8.56557302e-03  8.49253871e-03 -8.56557302e-03 ... -8.24492890e-03
   -8.56557302e-03  8.46440159e-03]
  [-8.47084913e-03  8.41196533e-03  8.17160308e-03 ...  8.32176488e-03
   -8.47084913e-03  8.41195788e-03]
  [ 1.81080468e-04  2.60910019e-05 -1.83196535e-04 ...  1.84235381e-04
    4.28961939e-05 -1.83196549e-04]
  ...
  [ 3.23750019e-01 -1.48956001e+00 -1.45888567e+00 ...  3.23748469e-01
    3.23750019e-01 -1.48956001e+00]
  [-1.08322275e+00 -5.59503555e-01 -1.09224725e+00 ... -1.10946000e+00
   -1.07296515e+00 -6.14310861e-01]
  [-1.15893252e-01 -4.64537442e-02 -2.19713330e-01 ... -1.00057796e-01
   -2.16928691e-01 -2.19713062e-01]]

 [[ 8.59982520e-03 -8.56557302e-03 -8.56557302e-03 ... -8.56536068e-03
    8.59982520e-03 -8.56557302e-03]
  [-8.47084913e-03  8.40512756e-03  7.56200776e-03 ... -8.47084913e-03
   -8.47084913e-03  8.40776879e-03]
  [ 9.62572376e-05 -1.54731970e-04 -1.83196535e-04 ...  1.84136341e-04
   -1.83196549e-04 -7.46708683e-05]
  ...
  [-1.48956001e+00 -1.4

In [26]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

cmap=plt.cm.get_cmap(plt.cm.viridis,143)


# plt.rcParams['font.size'] = 11.
# plt.rcParams['font.family'] = 'Comic Sans MS'
# plt.rcParams['axes.labelsize'] = 15.
# plt.rcParams['xtick.labelsize'] = 10.
# plt.rcParams['ytick.labelsize'] = 10.

plt.figure(figsize=(15,10))



sides = generate_signals
print(len(sides))

slip = 1200

# Peg_Position_x = sides[:,0]
Peg_Position_x = sides[:,0][:slip]
# Peg_Position_x = sides[:,0][0:10:slip]

# Peg_Position_y = sides[:,1]
Peg_Position_y = sides[:,1][:slip]
# Peg_Position_y = sides[:,1][0:10:slip]

# Peg_Position_z = sides[:,2]
Peg_Position_z = sides[:,2][:slip]
# Peg_Position_z = sides[:,2][0:10:slip]

# Angle = sides[:,3]
Angle = sides[:,3][:slip]
# Angle = sides[:,3][0:10:slip]

# Force_x = sides[:,4]
Force_x = sides[:,4][:slip]
# Force_x = sides[:,4][0:10:slip]

# Force_y = sides[:,5]
Force_y = sides[:,5][:slip]
# Force_y = sides[:,5][0:10:slip]

# Force_z = sides[:,6]
Force_z = sides[:,6][:slip]
# Force_z = sides[:,6][0:10:slip]

# Moment_x = sides[:,7]
Moment_x = sides[:,7][:slip]
# Moment_x = sides[:,7][0:10:slip]

# Moment_y = sides[:,8]
Moment_y = sides[:,8][:slip]
# Moment_y = sides[:,8][0:10:slip]

# Moment_z = sides[:,9]
Moment_z = sides[:,9][:slip]
# Moment_z = sides[:,9][0:10:slip]

x = np.linspace(0,len(Moment_z),len(Moment_z))

plt.plot(x, Force_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_x')
plt.plot(x, Force_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_y')
plt.plot(x, Force_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_z')

plt.plot(x, Moment_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_x')
plt.plot(x, Moment_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_y')
plt.plot(x, Moment_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_z')

plt.plot(x, Peg_Position_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_x')
plt.plot(x, Peg_Position_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_y')
plt.plot(x, Peg_Position_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_z')

plt.plot(x, Angle, marker='o', linestyle='-', markersize=2, linewidth=1, label='Angle')
# plt.plot(x, Counter, marker='o', linestyle='-', markersize=2, linewidth=1, label='Counter')


plt.xlabel('Times')
plt.ylabel('Signals')
plt.title('Force Torque Signals(3 Siles)')
plt.legend(loc='lower right')
plt.show()

110


ValueError: Image size of 896x158525 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1080x720 with 1 Axes>